In [102]:
import re
import pandas as pd
import numpy as np
import ast
import operator



def union_num_index_syn(dic_syn):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for syn in dic.values():
            synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_)    


def union_num_index_syn_rule_out(dic_syn,list_index_rule_out):
    _dic_syn_=dic_syn.copy()
    for row,dic in enumerate(_dic_syn_):
        synset=set()
        for idx,syn in dic.items():
            if idx in list_index_rule_out[row]:
                synset=synset.union(set(syn))
        _dic_syn_[row]=synset
    return(_dic_syn_) 

    
def union_syns(dataset,features,union_feature):
    _dataset_=dataset.copy()
    _dataset_[union_feature]=""
    for row in range(len(_dataset_[union_feature])):
        synset=set()
        for feature in features:
            synset=synset.union(set(_dataset_[feature][row]))
        _dataset_[union_feature][row]=synset
    return(_dataset_[union_feature])


def difference_cols(col_1,col_2):
    col_1_=col_1.copy()
    for row in range(0,len(col_1)):
        set_1=col_1[row]
        set_2=col_2[row]
        col_1_[row]=set_1.difference(set_2)
    return(col_1_)

def synonym_voter(dataset,features,union_and_removed_feature):
    _dataset_=dataset.copy()
    _dataset_[union_and_removed_feature]=_dataset_[union_and_removed_feature].apply(lambda s: list(s))
    _dataset_["temp"]=""
    for row in range(0,len(_dataset_[union_and_removed_feature])):
        ranked_dic={}
        for value in (_dataset_[union_and_removed_feature][row]):
            for feature in features:
                if value in _dataset_[feature][row]:
                    if value in ranked_dic:
                        ranked_dic[value]+=1
                    else:
                        ranked_dic[value]=1
        ranked_dic = sorted(ranked_dic.items(), key=operator.itemgetter(1),reverse=True)
        _dataset_["temp"][row]=ranked_dic
    return(_dataset_["temp"])
        

In [104]:
words_table=pd.read_excel(
    "/Users/mehdi.mirzapour/Dropbox/PractiKPharma/Lexicons/Word_Sense_Extraction_outcome_Update.xlsx")

columns=['JDM_Sense',
         'JDM',
         'French_WordNet',
         'Cnrtl',
         'Synonymo',
         'Cisco',
         'DicSyn_Sense']


words_table=words_table.fillna("[]")
words_table["DicSyn_Sense"]=words_table["DicSyn_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))
words_table["JDM_Sense"]=words_table["JDM_Sense"].apply(lambda s: re.sub("\[\]", '{}', s))

for col in columns:
    words_table[col]=words_table[col].apply(lambda s: re.sub(",[\s]*nan", ' ', s))
    words_table[col]=words_table[col].apply(lambda s: re.sub("[^,\s]*nan[\s]+,", '[', s))
    words_table[col]=words_table[col].apply(lambda s: ast.literal_eval(s))

words_table["DicSyn_Sense_"]=union_num_index_syn(words_table["DicSyn_Sense"])  
words_table["JDM_Sense_"]=union_num_index_syn(words_table["JDM_Sense"])  

words_table["Union_Sense"]=union_syns(words_table,
                                      ['JDM_Sense_','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_']
                                      ,"Union_Sense")

words_table["DicSyn_Sense_Filter"]="[]"
words_table["DicSyn_Sense_Filter"]=words_table["DicSyn_Sense_Filter"].apply(lambda s: ast.literal_eval(s))

words_table["DicSyn_Sense_to_be_del"]=union_num_index_syn_rule_out(words_table["DicSyn_Sense"],words_table["DicSyn_Sense_Filter"])  

words_table["JDM_Sense"]=words_table["Union_Sense"].apply(lambda s: s.difference())
words_table["Final_List"]=difference_cols(words_table["Union_Sense"],words_table["DicSyn_Sense_to_be_del"])

words_table["Synonyms_Voting"]=synonym_voter(words_table,
              ['JDM_Sense_','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_'],
              "Final_List")

words_table_to_write=words_table[['Word','Word_Corrected','JDM_Sense','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense','Synonyms_Voting']]

words_table_to_write.to_excel("/Users/mehdi.mirzapour/Dropbox/PractiKPharma/Lexicons/Word_Sense_Extraction_Voting.xlsx")

words_table_to_write  

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Word,Word_Corrected,JDM_Sense,JDM,French_WordNet,Cnrtl,Synonymo,Cisco,DicSyn_Sense,Synonyms_Voting
0,suffisant,suffisant,"{tolérable, contente, correct, présomptueux, p...","[important, vaniteux, correct, crâneur, raison...","[adéquat, suffisant, bégueule, collet monté, p...","[pourvoir, satisfaire, apaiser, baster, conten...","[acceptable, affecté, approuvable, arrogant, a...","[vaniteux, fat, orgueilleux, prétentieux, prés...","{0: ['arrogant', 'insolent', 'orgueilleux', 'p...","[(présomptueux, 5), (passable, 5), (arrogant, ..."
1,preuve,preuve,"{évidence, charge, indication, pièce, épreuve ...","[certificat, signe, raisonnement, document, il...","[corroboration, preuve, validation, salon, exe...","[témoignage, justification, marque, signe, ind...","[échantillon, affirmation, épreuve judiciaire,...","[témoignage, justification, indice, démonstrat...","{0: ['fait', 'charge', 'document', 'justificat...","[(indice, 6), (témoignage, 6), (constatation, ..."
2,nié,nié,"{disconvenir, disconvenu, réfuté, démenti, imp...","[désavoué, rejeté, dénié, renié, démenti, disc...",[],"[contester, contredire, désavouer, refuser, ré...","[confirmé, contesté, contester, contredire, co...",[],{},"[(disconvenir, 2), (disconvenu, 2), (disputer,..."
3,cause,cause,"{origine, effort, ferment, base, départ, final...","[origine, agent, occasion, facteur, raison, ar...","[cause, chose, objet, raison, campagne, effort...","[motif, principe, origine, raison, germe, fond...","[action, affaire, agent, élément, artisan, éti...","[motif, germe, principe, fondement, origine, p...","{0: ['affaire', 'dossier', 'procès', 'sujet', ...","[(motif, 6), (matière, 6), (raison, 6), (ferme..."
4,source,source,"{origine, ferment, début, base, départ, fonte ...","[base, agent, provenance, puits, foyer, fontai...","[source, graine, semence, puits, caractère, fa...","[origine, principe, racine, fondement, germe, ...","[agent, amont, étymologie, base, canal, centre...","[origine, racine, principe, germe, ferment, pr...","{0: ['commencement', 'début', 'origine', 'prin...","[(origine, 6), (puits, 6), (fontaine, 6), (pro..."
5,problèmes,problème,"{sale histoire, micmac, histoire, problèmes, e...",[],"[binz, bordel, bourbier, bousbir, embrouilles,...","[difficulté, ennui, question, affaire, histoir...","[affaire, énigme, complication, danger, diffic...",[],"{0: ['fait', 'difficulté', 'ennui', 'question'...","[(ennui, 3), (question, 3), (énigme, 3), (diff..."
6,raison,raison,"{origine, base, ratio, abord, cervelle, tact, ...","[cause, jugeote, logique, intellect, considéra...","[cause, raison, cas, complet, ensemble, poursu...","[esprit, sens, intelligence, sens, entendement...","[allégation, équité, argument, argumentation, ...","[entendement, motif, esprit, intelligence, jug...","{0: ['allégation', 'justification', 'excuse', ...","[(motif, 7), (cause, 6), (entendement, 6), (po..."
7,nie,nie,"{rejeter, désavouer, dénier, démonter, contest...",[],[],"[contester, contredire, désavouer, refuser, ré...","[confirmé, contesté, contester, contredire, co...","[dénier, réfuter, désavouer, contredire, démen...",{},"[(désavouer, 3), (démonter, 3), (disputer, 3),..."
8,exclu,exclu,"{renvoyé, licencié, excepté, exclusivement, ép...","[paria, renvoyé, expulsé, forclos, ôté, chassé...",[],"[exclusivement, expulsé, forclos, paria, renvoyé]","[exclusivement, expulsé, forclos, hippie, inen...","[forclos, renvoyé, exclusivement, expulsé, pro...","{0: ['exclusivement'], 1: ['forclos', 'renvoyé...","[(forclos, 5), (renvoyé, 4), (exclusivement, 4..."
9,histoire,histoire,"{bobard, chroniques, souvenirs, bible, archæol...","[roman, conte, affaire, biographie, chronologi...","[histoire, historique, passé, étage, chronique...","[conte, fable, récit, roman, affaire, mensonge...","[accroc, écho, affaire, affectation, allégorie...","[conte, fable, récit, roman, anecdote, bonimen...","{0: ['annales', 'archives', 'chroniques', 'fas...","[(récit, 7), (anecdote, 6), (conte, 6), (menso..."


In [3]:
words_table["JDM_Sense_Filters"]=[0]for i in ['JDM_Sense_','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_']:
    print([x for x in words_table["JDM"][0] if x not in words_table["Union_Sense"][0]]==[])

True
True
True
True
True
True
True


In [93]:
dataset=words_table
features=['JDM_Sense_','JDM','French_WordNet','Cnrtl','Synonymo','Cisco','DicSyn_Sense_']
union_and_removed_feature="Final_List"

_dataset_=dataset.copy()
_dataset_[union_and_removed_feature]=_dataset_[union_and_removed_feature].apply(lambda s: list(s))
for row in range(0,len(_dataset_[union_and_removed_feature])):
    ranked_dic={}
    for value in (_dataset_[union_and_removed_feature][row]):
        print(value)
        for feature in features:
            if value in _dataset_[feature][row]:

                if value in ranked_dic:
                    ranked_dic[value]+=1
                else:
                    ranked_dic[value]=1
        ranked_dic = sorted(ranked_dic.items(), key=operator.itemgetter(1),reverse=True)
        _dataset_[union_feature][row]=ranked_dic
    _dataset_[union_feature]


tolérable


NameError: name 'union_feature' is not defined

tolérable
contente
correct
passable
baster
raisonnable
conquérant
pourvoir
infatué
hautin
immodeste
assez
content
fournir
subvenir
compétent
apaiser
sotte
satisfaire
plastronneur
supérieur
adéquat
contentes
honorable
modéré
poseur
crâneur
suffisant
acceptable
doctoral
bégueule
vain
convenable
dédaigneux
assez bien
honnête
vainqueur
étant
strict
collet monté
superbe
affecté
approuvable
faraud
supportable
congru
hautain
pas mal
bon
contenter
supérieure
bien
satisfait
prude
satisfaisant
outrecuidant
bêcheur
décent
sot
glorieux
médiocre
évidence
indication
pièce
épreuve judiciaire
ordalie
élément
fondement
garantie
preuve
constatation
pierre de touche
corroboration
certificat
point
signe
validation
jugement de Dieu
empreinte
échantillon
raisonnement
établissement
corps du délit
justification
satisfecit
témoin
judiciaire
critère
confirmation
critérium
touche
exemple
motif
exercice
langue
argument
illustration
allusion
traces
caution
attestation
affirmation
enseigne
démonstration
Dieu
convic

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


origine
effort
ferment
base
départ
finalité
élément
instigateur
abord
semence
cas
fait
pourquoi
cause
chicane
mère
convention
déclencheur
occasion
incidence
chute
mobile
substance
motivation
condition
cobaye
faute
motif
campagne
chose
explication
action
rationalité
modalité
prendre
objet
inspiration
considération
causer
stipulation
étincelle
litige
conviction
promoteur
lieu
aitiologie
intention
artisan
conséquence
ætiologie
complet
crime
moyen
fondement
rôle
æthiologie
résultat
germe
inspirateur
poursuite judiciaire
espèce
principes
modération
affaires
auteur
effet
objets
pour
ensemble
rapport
créateur
ressort
intérêt
partie
prétexte
moteur
mouvement
étiologie
brandon
suite
excuse
défense
but
cuvelage
source
rapports
facteur
parti
agent
principe
raison
party
intellect
ferment
base
départ
fonte de caractères
semence
point
point d'eau
hydrologie
mère
face
geyser
point d
griffon
filet
étymologie
seuil
police d'écriture
instigatrice
texte
foyer
pépinière
Code source
puits
filon
eau
informa

argot
s'adresser
s'entretenir
apprendre
bégayer
pratiquer
ouvrir la bouche
toucher
discours
jargon
jabotter
sans
frapper
se mettre à table
patois
rabâcher
parlementer
articulation
giberner
citer
entretenir
énoncer
relater
s'abandonner
négociation
ordonner
déclamer
de
radoter
palabrer
prononciation
raconter
enregistrer
se proposer
énoncé
pour
disserter
débattre
jaboter
signaler
élocution
assujettir
crier
s'expliquer
articuler
pérorer
téléphoner
baragouiner
intercéder
accent
vider son sac
vernaculaire
s'épancher
marroner
usage
s'exprimer
stigmate
piste
couronne
renvoi
parafe
restes
cryptogramme
invitation
trempe
immeuble
expression
point
conclu
oeil
signal
totem
cachet
vestige
prodige
témoin
lettre
critère
accepté
graphie
distinctif
trait
abréviation
témoignage
allégorie
notation
souscrit
représentation
idéogramme
representation
contresigné
auspices
figure
pronostic
attestation
observation
pas
paraphe
manoir
manifestation
image
propre
chiffre
trace
tache
avant-coureur
sceau
substitut
nat

résister
rejeter
désobéir
renvoyer
remercier
recaler
boycotter
repousser
éliminer
excuser
regimber
répugner
envoyer
blackboulé
blackbouler
éconduire
rebiffer
défendre
transgresser
admissible
compte
étendre
décliner
répudier
battu
nier
contester
rebuter
dénier
rebeller
bouder
exclure
charge
avoir à
obligation
rôle
reste
loi
dette
de
nécessité
travail
exigence
probité
épreuve
penser
falloir
responsabilité
fonction
pensum
tâche
affaire
à
morale
civilité
soin
être obligé
exercice
contracter
composition
compagnonnage
obligé
interrogation
avoir besoin de
copie
avoir
être en reste
bien
tenir de
scrupule
service
office
appartenir
commandement
être tenu de
impératif
redevoir
hommage
respect
corvée
se rendre
chemin
aboutir
bout
rendre
faire du chemin
accoster
joindre
aborder
accomplir
rencontrer
réussir
se pousser
imposer
entrer
par
atteindre
percer
venir
égaler
accéder
déboucher
se hausser
toucher
obtenir
monter
gagner
arriver
élever
pousser
hausser
finir par
papa
parent
procréateur
pépé
ancien

...

siecle
conséquence
passage
endroit
cru
incertain
préjuger
regardé comme
auguré
regardé
pressentir
officiel
pressenti
espérer
préjugé
prétendre
putatif
prévu
pour
estimé
regarder
probable
soupçonné
présupposer
présagé
pensé
compté
conjecturé
présomptif
censé
compter
attendre
hypothétique
espéré
présupposé
réputé
présager
soi-disant
supposé
réputer
attendu
surestimé
prétendu
début
automne
jeune
renaissance
renouvellement
réapparition
pige
recommencement
du printemps
source
ans
progrès
renouveau
avril
saison
printemps
jeunesse
belle saison
âge
Sept
septembre
juin
Juin
messidor
prairial
cru
redouter
suspect
méfier
de
sentir
état mental
défier
pressenti
penser
subodoré
imaginer
redouté
croire
subodorer
conjecturer
entrevu
deviné
suspecté
flairé
douté
pensé
imaginé
conjecturé
senti
supposé
suspecter
présumé
Université Harvard
décembre
mammy
grand
mamy
bonne-maman
mémère
mère-grand
grannie
grand'maman
grand-maman
maman
m'amie
mamita
vieille
aïeul
grand-mère
déshabiller
imprimer
prodiguer
dépl

tantouse
tapette
oncle
parent
amitat
prout
sodomite
tatie
tante
PD
pédale
lope
agréer
sauter
chemin
ressortir
passer
renaître
regagner
afférer
sortir
rallier
remettre
convenir
reprendre
retourner
échoir
équivaloir
réexaminer
réintégrer
reconsidérer
retrouver
incomber
valoir
rappliquer
réveiller
réservé
récidiver
relever
reparaître
reporter
satisfaire
rabattre
tirer
coûter
repasser
repousser
ramener
retomber
récupérer
refluer
reproduire
apparaître
réconcilier
plaire
obtenir
rétrograder
reculer
ressurgir
occuper
tour
monter
rentrer
reparler
guérir
appartenir
rétablir
bride
contremander
réapparaître
recommencer
reconduire
rejoindre
antécédence
implication
rétroacte
passé
antériorité
hérédité
condition
enthymème
ancienneté
préliminaire
précurseur
antan
temps
application
conséquence
exemple
avril
printemps
Avril
germinal
aurore
flottement
soupçon
incroyance
vacillation
question
méfiance
obscur
inquiétude
jalousie
aucun
mettre en doute
indétermination
suspicion
irrésolution
perplexité
doute


boulonné
imbriqué
accordé
intime
relié
lié
conjuguer
agencé
résidu
ligoter
tenir
sédiment
rattaché
uni
river
lacer
fèces
rapprocher
joindre
agglutiner
allier
dépôt
corréler
botteler
coordonner
bottelé
coordonné
rattacher
cimenté
rivé
fusionné
analogique
assujettir
racaille
adhérent
contraindre
associer
obligé
charpenté
relier
obliger
familier
astreindre
astreint
garrotté
solidariser
nouer
amarré
accoupler
parcelle
un
soupçon
larme
grain
méfiance
suspect
nuage
goutte
simulacre
suspicion
doute
défiance
apparence
pointe
destruction
scepticisme
opinion
peu
pincée
ombrage
nuance
croyance
semblant
idée
gramme
crainte
teinte
supposition
miette
ombre
brin
rien
atome
dans
un
automne
en
cycle
par
années
an
age
date
balai
temps
période
à
une
hyver
année
débiliter
dessécher
déshydrater
drainer
épuiser
affaiblir
attirer
sécher
pomper
conduire
imbiber
étancher
abattre
baigner
doucher
puiser
égoutter
arroser
décrire
déshabiller
trahir
démasquer
masque
brûler
lever
étaler
déceler
deviner
montrer
livre

déclamé
stipulé
formel
répondu
dévoiler
confier
confessé
indiqué
énoncé
déposer
proclamé
énuméré
signaler
découvert
communiquer
manifesté
jurer
dénoncé
confesser
expliqué
connaissance
protester
certain
prononcer
énumérer
état
proclamer
manifester
certifié
publier
affirmer
publié
prétendu
informé
trou
aurore
vous
époque
clarté
goulet
angle
moment
trouée
nuit
lueur
fenêtre
fissure
tournure
page
journée
rétribution
extérieur
éclairage
vie
aspect
matin
date
interstice
lumière
siecle
cratère
crevé
judas
silhouette
perspective
férie
naissance
caractère
moyen
vue
durée
ajour
siècle
luminosité
ouverture
semblance
combientième
traitement
apparence
crépuscule
période
échappée
heure
brèche
lever
existence
passe
quantième
soleil
optique
temps
aube
fente
passage
âge
original
proche
contemporain
inédit
nouvelle
évidence
indication
pièce
épreuve judiciaire
ordalie
fondement
garantie
constatation
pierre de touche
certificat
signe
jugement de Dieu
empreinte
échantillon
raisonnement
établissement
corps 

promener
admis
passé
autoriser
décédée
affadir
tourner
remettre
rallier
poindre
lâcher
prêter
enfuir
excéder
résigner
croiser
mourir
entrer
accepter
refiler
subir
transformer
cribler
communiquer
conclure
contourner
éclat
outrepasser
éventer
passe
transférer
mettre
vivre
contenter
tolérer
laisser
filtrer
appliquer
plonger
recevoir
courir
souffrir
vêtir
éclairer
laver
enduire
traverser
enfoncer
dérouler
soupir
suivre
engouffrer
transiter
supporter
expirer
et
s
revenir
écarter
écouler
avarier
faufiler
enjamber
escalader
imbiber
baigner
déteindre
effacer
évaporer
faire
éteindre
ternir
pénétrer
finir
infiltrer
satisfaire
flétrir
agoniser
violer
percer
consacrer
tuer
ancienne
envoler
surpasser
avérer
être
dépasser
se présenter
reçu
décéder
lotionner
occuper
être digéré
trace
admettre
glisser
évanouir
marcher
éclaircir
doubler
pardonner
rejoindre
transmettre
remplir
sauter
moisir
digérer
ressortir
côtoyer
évoluer
cesser
jouer
joindre
sortir
de
parvenir
le
transfuser
tamiser
circuler
à
fourrer

In [97]:
x[0]

[('passable', 5),
 ('convenable', 5),
 ('raisonnable', 4),
 ('crâneur', 4),
 ('honnête', 4),
 ('supportable', 4),
 ('satisfaisant', 4),
 ('tolérable', 3),
 ('correct', 3),
 ('conquérant', 3),
 ('infatué', 3),
 ('assez', 3),
 ('honorable', 3),
 ('acceptable', 3),
 ('superbe', 3),
 ('congru', 3),
 ('bon', 3),
 ('satisfait', 3),
 ('outrecuidant', 3),
 ('immodeste', 2),
 ('compétent', 2),
 ('plastronneur', 2),
 ('supérieur', 2),
 ('poseur', 2),
 ('suffisant', 2),
 ('doctoral', 2),
 ('vain', 2),
 ('strict', 2),
 ('pas mal', 2),
 ('bêcheur', 2),
 ('sot', 2),
 ('glorieux', 2),
 ('médiocre', 2),
 ('contente', 1),
 ('baster', 1),
 ('pourvoir', 1),
 ('hautin', 1),
 ('content', 1),
 ('fournir', 1),
 ('subvenir', 1),
 ('apaiser', 1),
 ('sotte', 1),
 ('satisfaire', 1),
 ('adéquat', 1),
 ('contentes', 1),
 ('modéré', 1),
 ('bégueule', 1),
 ('dédaigneux', 1),
 ('assez bien', 1),
 ('vainqueur', 1),
 ('étant', 1),
 ('collet monté', 1),
 ('affecté', 1),
 ('approuvable', 1),
 ('faraud', 1),
 ('hautain', 